# Compile the Data

The source data is maintained in a relational database with each table provided as an individual json. The following code is going to pull in the necessary tables to populate our dataframe. It will then perform cleaning and reformatting of some fields to create a master copy of all the relevant data. This will allow me to query based on league, team, and players to get the focused data I need for visualizations and predictions. Each of those processes will further modify the dataframe to account for the end goal.

## 1. Create the primary dataframe
#### 1.1 Events data
#### 1.2 Match data
#### 1.3 Join Match and Event data
## 2. Supporting tables
#### 2.1 Team data
#### 2.2 Player data
#### 2.3 Competition data
## 3. Clean-up the final dataframe
#### 3.1 Create coordinate columns
#### 3.2 Create positional grids
#### 3.3 Address missing values
## 4. Export to csv

In [1]:
import pandas as pd
import requests
import json
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 1. Create the primary dataframe

In [2]:
# a function to compile the tables for all the different competitions into a single dataframe
def buildDF(dataframe, jsons):
    for i in jsons:
        print(i)
        temp = pd.read_json(i)
        frame = [temp, dataframe]
        dataframe = pd.concat(frame)
        
    return dataframe

In [3]:
# create initial dataframes for events and matches using the England data
events= pd.read_json('events_England.json')
matches= pd.read_json('matches_England.json')

### 1.1 Events data

In [4]:
# compile all the events data from the various leagues
leagues = ['events_France.json',
           'events_Germany.json',
           'events_Italy.json',
           'events_Spain.json']

events = buildDF(events, leagues)

events_France.json
events_Germany.json
events_Italy.json
events_Spain.json


In [5]:
events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422


In [6]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3071395 entries, 0 to 643149
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   eventId       int64  
 1   subEventName  object 
 2   tags          object 
 3   playerId      int64  
 4   positions     object 
 5   matchId       int64  
 6   eventName     object 
 7   teamId        int64  
 8   matchPeriod   object 
 9   eventSec      float64
 10  subEventId    object 
 11  id            int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 304.6+ MB


### 1.2 Match data

In [7]:
# compile all the match data from the various leagues
leagueMatches = ['matches_England.json',
                 'matches_France.json',
                 'matches_Germany.json',
                 'matches_Italy.json',
                 'matches_Spain.json']


matches= buildDF(matches, leagueMatches)

matches_England.json
matches_France.json
matches_Germany.json
matches_Italy.json
matches_Spain.json


In [8]:
matches.head()

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4406122,38,"{'676': {'scoreET': 0, 'coachId': 92894, 'side...",181144,2018-05-20 18:45:00,676,Camp Nou,2565922,"Barcelona - Real Sociedad, 1 - 0","May 20, 2018 at 8:45:00 PM GMT+2","[{'refereeId': 398931, 'role': 'referee'}, {'r...",Regular,795
1,Played,4406122,38,"{'679': {'scoreET': 0, 'coachId': 3427, 'side'...",181144,2018-05-20 16:30:00,0,Estadio Wanda Metropolitano,2565925,"Atl\u00e9tico Madrid - Eibar, 2 - 2","May 20, 2018 at 6:30:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795
2,Played,4406122,38,"{'691': {'scoreET': 0, 'coachId': 444778, 'sid...",181144,2018-05-20 14:15:00,691,San Mam\u00e9s Barria,2565919,"Athletic Club - Espanyol, 0 - 1","May 20, 2018 at 4:15:00 PM GMT+2","[{'refereeId': 384957, 'role': 'referee'}, {'r...",Regular,795
3,Played,4406122,38,"{'674': {'scoreET': 0, 'coachId': 210074, 'sid...",181144,2018-05-20 10:00:00,674,Estadio de Mestalla,2565924,"Valencia - Deportivo La Coru\u00f1a, 2 - 1","May 20, 2018 at 12:00:00 PM GMT+2","[{'refereeId': 398913, 'role': 'referee'}, {'r...",Regular,795
4,Played,4406122,38,"{'675': {'scoreET': 0, 'coachId': 275283, 'sid...",181144,2018-05-19 18:45:00,0,Estadio de la Cer\u00e1mica,2565927,"Villarreal - Real Madrid, 2 - 2","May 19, 2018 at 8:45:00 PM GMT+2","[{'refereeId': 395085, 'role': 'referee'}, {'r...",Regular,795


In [9]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2206 entries, 0 to 379
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   status         2206 non-null   object
 1   roundId        2206 non-null   int64 
 2   gameweek       2206 non-null   int64 
 3   teamsData      2206 non-null   object
 4   seasonId       2206 non-null   int64 
 5   dateutc        2206 non-null   object
 6   winner         2206 non-null   int64 
 7   venue          2206 non-null   object
 8   wyId           2206 non-null   int64 
 9   label          2206 non-null   object
 10  date           2206 non-null   object
 11  referees       2206 non-null   object
 12  duration       2206 non-null   object
 13  competitionId  2206 non-null   int64 
dtypes: int64(6), object(8)
memory usage: 258.5+ KB


In [10]:
# change the wyId column to matchID to allow for joining the match data to the event data
matches.rename({'wyId': 'matchId'}, axis=1, inplace=True)

In [11]:
matches[matches.gameweek == 27]

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,matchId,label,date,referees,duration,competitionId
110,Played,4406122,27,"{'692': {'scoreET': 0, 'coachId': 3880, 'side'...",181144,2018-03-05 20:00:00,692,Estadio de Bala\u00eddos,2565813,"Celta de Vigo - Las Palmas, 2 - 1","March 5, 2018 at 9:00:00 PM GMT+1","[{'refereeId': 395078, 'role': 'referee'}, {'r...",Regular,795
111,Played,4406122,27,"{'674': {'scoreET': 0, 'coachId': 0, 'side': '...",181144,2018-03-04 19:45:00,674,Estadio de Mestalla,2565811,"Valencia - Real Betis, 2 - 0","March 4, 2018 at 8:45:00 PM GMT+1","[{'refereeId': 378950, 'role': 'referee'}, {'r...",Regular,795
112,Played,4406122,27,"{'687': {'scoreET': 0, 'coachId': 4784, 'side'...",181144,2018-03-04 17:30:00,687,Estadio Municipal de Anoeta,2565815,"Real Sociedad - Deportivo Alav\u00e9s, 2 - 1","March 4, 2018 at 6:30:00 PM GMT+1","[{'refereeId': 378994, 'role': 'referee'}, {'r...",Regular,795
113,Played,4406122,27,"{'676': {'scoreET': 0, 'coachId': 92894, 'side...",181144,2018-03-04 15:15:00,676,Camp Nou,2565817,"Barcelona - Atl\u00e9tico Madrid, 1 - 0","March 4, 2018 at 4:15:00 PM GMT+1","[{'refereeId': 381927, 'role': 'referee'}, {'r...",Regular,795
114,Played,4406122,27,"{'691': {'scoreET': 0, 'coachId': 169425, 'sid...",181144,2018-03-04 11:00:00,0,Estadio Ciudad de Valencia,2565816,"Levante - Espanyol, 1 - 1","March 4, 2018 at 12:00:00 PM GMT+1","[{'refereeId': 398945, 'role': 'referee'}, {'r...",Regular,795
115,Played,4406122,27,"{'675': {'scoreET': 0, 'coachId': 275283, 'sid...",181144,2018-03-03 19:45:00,675,Estadio Santiago Bernab\u00e9u,2565812,"Real Madrid - Getafe, 3 - 1","March 3, 2018 at 8:45:00 PM GMT+1","[{'refereeId': 420995, 'role': 'referee'}, {'r...",Regular,795
116,Played,4406122,27,"{'712': {'scoreET': 0, 'coachId': 5326, 'side'...",181144,2018-03-03 17:30:00,712,Estadio Municipal de Butarque,2565808,"Legan\u00e9s - M\u00e1laga, 2 - 0","March 3, 2018 at 6:30:00 PM GMT+1","[{'refereeId': 385473, 'role': 'referee'}, {'r...",Regular,795
117,Played,4406122,27,"{'677': {'scoreET': 0, 'coachId': 0, 'side': '...",181144,2018-03-03 17:30:00,0,Estadio Municipal de Riazor,2565810,"Deportivo La Coru\u00f1a - Eibar, 1 - 1","March 3, 2018 at 6:30:00 PM GMT+1","[{'refereeId': 381854, 'role': 'referee'}, {'r...",Regular,795
118,Played,4406122,27,"{'678': {'scoreET': 0, 'coachId': 292972, 'sid...",181144,2018-03-03 15:15:00,680,Estadio Ram\u00f3n S\u00e1nchez Pizju\u00e1n,2565814,"Sevilla - Athletic Club, 2 - 0","March 3, 2018 at 4:15:00 PM GMT+1","[{'refereeId': 384923, 'role': 'referee'}, {'r...",Regular,795
119,Played,4406122,27,"{'756': {'scoreET': 0, 'coachId': 167044, 'sid...",181144,2018-03-03 12:00:00,756,Estadio de la Cer\u00e1mica,2565809,"Villarreal - Girona, 0 - 2","March 3, 2018 at 1:00:00 PM GMT+1","[{'refereeId': 460924, 'role': 'referee'}, {'r...",Regular,795


### 1.3 Join matches and events

In [12]:
soccer = pd.merge(matches, events, on=['matchId'], how='outer')

In [13]:
soccer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3714545 entries, 0 to 3714544
Data columns (total 25 columns):
 #   Column         Dtype  
---  ------         -----  
 0   status         object 
 1   roundId        int64  
 2   gameweek       int64  
 3   teamsData      object 
 4   seasonId       int64  
 5   dateutc        object 
 6   winner         int64  
 7   venue          object 
 8   matchId        int64  
 9   label          object 
 10  date           object 
 11  referees       object 
 12  duration       object 
 13  competitionId  int64  
 14  eventId        int64  
 15  subEventName   object 
 16  tags           object 
 17  playerId       int64  
 18  positions      object 
 19  eventName      object 
 20  teamId         int64  
 21  matchPeriod    object 
 22  eventSec       float64
 23  subEventId     object 
 24  id             int64  
dtypes: float64(1), int64(10), object(14)
memory usage: 736.8+ MB


## 2. Supporting tables
Loading the json files that represent teams, competitions, and players. These have corresponding id's that can be used to replace those reference values to the actual names.

In [14]:
import codecs

#Load the teams json file
teams= pd.read_json(codecs.open('teams.json', 'r','unicode_escape'))
#Load the competitions json file
competitions = pd.read_json('competitions.json')
#Load player json file
players = pd.read_json(codecs.open('players.json', 'r','unicode_escape'))

### 2.1 Team data

In [15]:
teams.head()

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [16]:
#we only need the team name
teams.drop(['city','officialName','area','type'], axis=1, inplace=True)

# rename wyId to teamId
teams.rename({'wyId': 'teamId'}, axis=1, inplace=True)

In [17]:
teams.head()

,name,teamId
0,Newcastle United,1613
1,Celta de Vigo,692
2,Espanyol,691
3,Deportivo Alavés,696
4,Levante,695


In [18]:
teams.name.unique()

array(['Newcastle United', 'Celta de Vigo', 'Espanyol',
       'Deportivo Alavés', 'Levante', 'Troyes', 'Getafe',
       "Borussia M'gladbach", 'Huddersfield Town', 'Athletic Club',
       'Atlético Madrid', 'Olympique Lyonnais', 'PSG', 'Valencia',
       'Real Madrid', 'Barcelona', 'Las Palmas', 'Leganés', 'SPAL',
       'Swansea City', 'Olympique Marseille', 'Nantes', 'Nice', 'Rennes',
       'Strasbourg', 'Eibar', 'AFC Bournemouth', 'Brighton & Hove Albion',
       'Werder Bremen', 'Bayer Leverkusen', 'Borussia Dortmund',
       'Bayern München', 'Stuttgart', 'Schalke 04', 'Milan', 'Angers',
       'Juventus', 'Roma', 'Sassuolo', 'Burnley', 'Bordeaux',
       'Hannover 96', 'Dijon', 'Hertha BSC', 'Wolfsburg', 'Hamburger SV',
       'Freiburg', 'Bologna', 'Metz', 'Sampdoria', 'Chievo', 'Lazio',
       'Udinese', 'Internazionale', 'Leicester City', 'West Ham United',
       'Stoke City', 'Benevento', 'Saint-Étienne', 'Girona', 'Watford',
       'Hoffenheim', 'Cagliari', 'Atalanta', 'F

In [19]:
# creating a dictionary to use replace teamIds with names in the primary dataframe
teamName = teams.set_index('teamId').to_dict()['name']

# use the dictionary to remap the teamId and winner to the team names from the teams dataframe
soccer.replace({'teamId':teamName}, inplace=True)
soccer.replace({'winner':teamName}, inplace=True)

### 2.2 Players data

In [20]:
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konaté,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konaté,null


In [21]:
# pull out just the first position code from the role column
players['position'] = [x['code2'] for x in players['role']]

# we only need the player's name, role, and wyId
players= players[['lastName','position','wyId']]

# rename wyId to teamId and position to role
players.rename({'wyId': 'playerId','position': 'role'}, axis=1, inplace=True)

In [22]:
len(players.lastName.unique())

3414

In [23]:
# merge the player data in with the soccer dataframe
soccer = pd.merge(soccer, players, on=['playerId'], how='outer')
#drop the playerId column
soccer.drop(['playerId'], axis=1, inplace=True)

### 2.3 Competition data

In [24]:
competitions.head()

,name,wyId,format,area,type
0,Italian first division,524,Domestic league,"{'name': 'Italy', 'id': '380', 'alpha3code': '...",club
1,English first division,364,Domestic league,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
2,Spanish first division,795,Domestic league,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,French first division,412,Domestic league,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
4,German first division,426,Domestic league,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club


In [25]:
# we only need the name and wyId
competitions.drop(['format','area','type'], axis=1, inplace=True)

# rename wyId to teamId
competitions.rename({'wyId': 'competitionId'}, axis=1, inplace=True)

In [26]:
# creating a dictionary to use replace teamIds with names in the primary dataframe
competitionName = competitions.set_index('competitionId').to_dict()['name']

# use the dictionary to remap the competition name to the competitionId in the soccer dataframe
soccer.replace({'competitionId':competitionName}, inplace=True)

## 3. Clean up the final dataframe

We will remove the columns that are not necessary to my analysis, and update the positions column to create a specific x/y coordinate columns relative to the field grid.

In [27]:
# drop the columns that we don't need
soccer.drop(['status','roundId','seasonId','dateutc','venue',
             'matchId','label','date','referees','duration',
             'id','tags','teamsData'], axis=1, inplace=True)

Due to the fact that this dataset includes international players from the World Cup who do not play in the European leagues we are querying, we will have some N/A values. I am just going to drop these before moving forward.

In [28]:
soccer.dropna(inplace=True)

In [29]:
soccer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440476 entries, 0 to 3714544
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   gameweek       float64
 1   winner         object 
 2   competitionId  object 
 3   eventId        float64
 4   subEventName   object 
 5   positions      object 
 6   eventName      object 
 7   teamId         object 
 8   matchPeriod    object 
 9   eventSec       float64
 10  subEventId     object 
 11  lastName       object 
 12  role           object 
dtypes: float64(3), object(10)
memory usage: 367.5+ MB


### 3.1 Create coordinate columns

In [30]:
# convert position into an x and y column for plotting purposes
# retrieved from: https://github.com/Friends-of-Tracking-Data-FoTD/mapping-match-events-in-Python/blob/master/data_exploration.ipynb

soccer['x'] = [x[0]['x'] for x in soccer['positions']]
soccer['y'] = [x[0]['y'] for x in soccer['positions']]

# update the x/y records to account for the fact they are a percentage of the field. 80% on y and 120% on x
soccer['x'] = (soccer['x']/100)*120
soccer['y'] = (soccer['y']/100)*80

In [31]:
soccer.describe()

,gameweek,eventId,eventSec,x,y
count,3.440476e+06,3.440476e+06,3.440476e+06,3.440476e+06,3.440476e+06
mean,1.926766e+01,5.694628e+00,1.395034e+03,5.874610e+01,3.997302e+01
std,1.079841e+01,3.141586e+00,8.357973e+02,2.962486e+01,2.423663e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,-1.200000e+00,0.000000e+00
25%,1.000000e+01,1.000000e+00,6.644043e+02,3.600000e+01,1.840000e+01
50%,1.900000e+01,8.000000e+00,1.380117e+03,5.640000e+01,4.000000e+01
75%,2.900000e+01,8.000000e+00,2.115222e+03,8.160000e+01,6.160000e+01
max,3.800000e+01,1.000000e+01,3.537356e+03,1.200000e+02,8.080000e+01


We have at least one record where the x-value is negative and one where the y-value is greater than 80. We need to fix this.

In [32]:
soccer[soccer.x < 0]

,gameweek,winner,competitionId,eventId,subEventName,positions,eventName,teamId,matchPeriod,eventSec,subEventId,lastName,role,x,y
909196,9.0,Lazio,Italian first division,7.0,Touch,"[{'y': 47, 'x': -1}, {'y': 12, 'x': 69}]",Others on the ball,Lazio,2H,2753.190257,72,Strakosha,GK,-1.2,37.6


In [33]:
soccer[soccer.y > 80]

,gameweek,winner,competitionId,eventId,subEventName,positions,eventName,teamId,matchPeriod,eventSec,subEventId,lastName,role,x,y
1570071,21.0,0,German first division,8.0,Simple pass,"[{'y': 101, 'x': 28}, {'y': 95, 'x': 18}]",Pass,Hertha BSC,1H,1660.632434,85,Pekarík,DF,33.6,80.8


In [34]:
soccer.loc[(soccer['x'] < 0), 'x'] = 0
soccer.loc[(soccer['y'] > 80), 'y'] = 80

### 3.2 Create positional grids

We will grid out the soccer pitch into 12 sections and assign each event to a section 0-15 based on the coordinates.

#### xGrids:         
####                       0-30
####                     31-60
####                     61-90
####                     91-120
#### yGrids:         
####                       0-20
####                     21-40
####                     41-60
####                     61-80

In [35]:
soccer["xGrid"] = np.nan
soccer["yGrid"] = np.nan
soccer["positionGrid"] = np.nan

# create four categories for positional x
soccer.loc[(soccer['x'] <= 30), 'xGrid'] = 0
soccer.loc[(soccer['x'] > 30) & (soccer['x'] <= 60), 'xGrid'] = 1
soccer.loc[(soccer['x'] > 60) & (soccer['x'] <= 90), 'xGrid'] = 2
soccer.loc[(soccer['x'] > 90) & (soccer['x'] <= 120), 'xGrid'] = 3

# create four categories for positional y
soccer.loc[(soccer['y'] <= 20), 'yGrid'] = 0
soccer.loc[(soccer['y'] > 20) & (soccer['y'] <= 40), 'yGrid'] = 1
soccer.loc[(soccer['y'] > 40) & (soccer['y'] <= 60), 'yGrid'] = 2
soccer.loc[(soccer['y'] > 60) & (soccer['y'] <= 80), 'yGrid'] = 3

In [36]:
# populate the positional categories in the positionGrid field
soccer.loc[(soccer['xGrid'] == 0) & (soccer['yGrid'] == 0), 'positionGrid'] = 0
soccer.loc[(soccer['xGrid'] == 0) & (soccer['yGrid'] == 1), 'positionGrid'] = 1
soccer.loc[(soccer['xGrid'] == 0) & (soccer['yGrid'] == 2), 'positionGrid'] = 2
soccer.loc[(soccer['xGrid'] == 0) & (soccer['yGrid'] == 3), 'positionGrid'] = 3
soccer.loc[(soccer['xGrid'] == 1) & (soccer['yGrid'] == 0), 'positionGrid'] = 4
soccer.loc[(soccer['xGrid'] == 1) & (soccer['yGrid'] == 1), 'positionGrid'] = 5
soccer.loc[(soccer['xGrid'] == 1) & (soccer['yGrid'] == 2), 'positionGrid'] = 6
soccer.loc[(soccer['xGrid'] == 1) & (soccer['yGrid'] == 3), 'positionGrid'] = 7
soccer.loc[(soccer['xGrid'] == 2) & (soccer['yGrid'] == 0), 'positionGrid'] = 8
soccer.loc[(soccer['xGrid'] == 2) & (soccer['yGrid'] == 1), 'positionGrid'] = 9
soccer.loc[(soccer['xGrid'] == 2) & (soccer['yGrid'] == 2), 'positionGrid'] = 10
soccer.loc[(soccer['xGrid'] == 2) & (soccer['yGrid'] == 3), 'positionGrid'] = 11
soccer.loc[(soccer['xGrid'] == 3) & (soccer['yGrid'] == 0), 'positionGrid'] = 12
soccer.loc[(soccer['xGrid'] == 3) & (soccer['yGrid'] == 1), 'positionGrid'] = 13
soccer.loc[(soccer['xGrid'] == 3) & (soccer['yGrid'] == 2), 'positionGrid'] = 14
soccer.loc[(soccer['xGrid'] == 3) & (soccer['yGrid'] == 3), 'positionGrid'] = 15

In [37]:
# convert our positionGrid to categorical
soccer["positionGrid"] = soccer["positionGrid"].astype('category')

In [38]:
# drop the yGrid and xGrid series as they were just to populate the positionGrid
soccer.drop(['xGrid', 'yGrid', 'positions'], axis=1, inplace=True)

### 3.3 Address missing values

I think that we do have a problem with eventId = "Offside". I do not think that this includes a subevent and as a result we will end up with null values. If we look at the unique values for subevents we see a ''. This is Offsides, I am going to populate this value to equate the the event ID

In [39]:
soccer.subEventId.unique()

array([85, 72, 13, 20, 11, 12, '', 10, 86, 100, 70, 23, 36, 82, 80, 83,
       21, 84, 71, 24, 31, 22, 30, 33, 32, 26, 35, 25, 34, 91, 40, 81, 90,
       27, 50, 51], dtype=object)

In [40]:
soccer.loc[93]

gameweek                             36
winner                          Sevilla
competitionId    Spanish first division
eventId                               6
subEventName                           
eventName                       Offside
teamId                    Real Sociedad
matchPeriod                          2H
eventSec                        1858.39
subEventId                             
lastName                  Jiménez López
role                                 FW
x                                  88.8
y                                    44
positionGrid                         10
Name: 93, dtype: object

In [41]:
soccer.loc[(soccer['eventId'] == 6), 'subEventName'] = 'Offside'
soccer.loc[(soccer['eventId'] == 6), 'subEventId'] = 6

In [42]:
soccer.isna().sum()

gameweek         0
winner           0
competitionId    0
eventId          0
subEventName     0
eventName        0
teamId           0
matchPeriod      0
eventSec         0
subEventId       0
lastName         0
role             0
x                0
y                0
positionGrid     0
dtype: int64

## 4. Export to csv

In [43]:
# Export our data frame to a csv to utilize in our analysis
soccer.to_csv('soccer.csv', index=False)

In [44]:
soccer.head()

,gameweek,winner,competitionId,eventId,subEventName,eventName,teamId,matchPeriod,eventSec,subEventId,lastName,role,x,y,positionGrid
0,38.0,Barcelona,Spanish first division,8.0,Simple pass,Pass,Real Sociedad,1H,1.005442,85,Jiménez López,FW,58.8,40.8,6.0
1,38.0,Barcelona,Spanish first division,8.0,Simple pass,Pass,Real Sociedad,1H,26.009290,85,Jiménez López,FW,86.4,67.2,11.0
2,38.0,Barcelona,Spanish first division,8.0,Simple pass,Pass,Real Sociedad,1H,97.700752,85,Jiménez López,FW,67.2,59.2,10.0
3,38.0,Barcelona,Spanish first division,8.0,Simple pass,Pass,Real Sociedad,1H,132.889252,85,Jiménez López,FW,68.4,66.4,11.0
4,38.0,Barcelona,Spanish first division,8.0,Simple pass,Pass,Real Sociedad,1H,265.013504,85,Jiménez López,FW,98.4,75.2,15.0
